<a href="https://colab.research.google.com/github/Seferp/diagram_generator/blob/main/diagram_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import plotly.express as px
import os
from shutil import move, rmtree, make_archive
from google.colab import files
import zipfile


class diagram():
    def __init__(self, main_path = '/content',
                 input_path = '/content/data_input/',
                 input_raw_path = '/content/raw_data_input/',
                 input_convert_path = '/content/convert_data_input',
                 input_csv_path = '/content/data_csv_input/',
                 output_path ='/content/data_output/'):
        self.main_path = main_path
        self.input_path = input_path
        self.input_raw_path = input_raw_path
        self.input_convert_path = input_convert_path
        self.input_csv_path = input_csv_path
        self.output_path = output_path


    def exist_check(self, path_to_check):
        if not os.path.exists(path_to_check):
            os.mkdir(path_to_check)

    #Download file from local disk ans save them in the main path
    def import_files_from_disc(self):
        files.upload()

    #Move files from main path to raw_data_input
    def move_files_to_raw_input_data(self):
        file_list = os.listdir(self.main_path)
        self.exist_check(self.input_raw_path)

        for file in file_list:
            if file.endswith('txt'):
                file_path = os.path.join(self.main_path, file)
                move(file_path, self.input_raw_path)


    #Convert .txt files to .csv files
    def convert_txt_file_to_csv(self):
        files_list = os.listdir(self.input_raw_path)
        self.exist_check(self.input_convert_path)
        self.exist_check(self.input_csv_path)

        for file in files_list:
            print(file)
            if file.endswith('txt'):
                file_path = os.path.join(self.input_raw_path, file)
                with open(file_path, 'r') as f:
                    raw_data = f.readlines()
                convert_file = os.path.join(self.input_convert_path, file)
                with open (convert_file, 'x') as f:
                    for line in raw_data[35:]:
                        split_line = line.split('	')
                        f.write(','.join(split_line))


            convert_txt_file_path = os.path.join(self.input_convert_path, file)
            csv_file_name = file.replace('txt', 'csv')
            convert_csv_file_path = os.path.join(self.input_csv_path, csv_file_name)

            convert_file_read = pd.read_csv(convert_txt_file_path)
            convert_file_read.to_csv(convert_csv_file_path, index= None)

    #Saves .csv files in data_input
    def saves_csv(self):
        files_list = os.listdir(self.input_csv_path)
        self.exist_check(self.input_path)

        for file in files_list:
            file_path = os.path.join(self.input_csv_path, file)
            convert_file_path = os.path.join(self.input_path, file)
            df = pd.read_csv(file_path)
            df = df[df.Status == 'RUNNING']
            df = df[['Time', 'FTAVG']]
            df = df.rename(columns = {'FTAVG': 'Diameter'})
            df['Diameter'] = df['Diameter'] / 1000
            df = df[df['Diameter'] > 0]

            for idx in range(1, len(df)):
                if df.iloc[idx]['Time'] < df.iloc[idx-1]['Time']:
                    df.iloc[idx, df.columns.get_loc('Time')] = df.iloc[idx-1]['Time'] + 1

            df.to_csv(convert_file_path, index = None)

    #Generate graphs based on the converted in previous step .csv files
    def generating_graphs(self):
        files_list = os.listdir(self.input_path)
        self.exist_check(self.output_path)

        for file in files_list:
            if file.endswith('csv'):
                file_path = os.path.join(self.input_path, file)
                df = pd.read_csv(file_path)
                df['min_value'] = 2.75
                df['max_value'] = 2.95
                max_y_value = df['Diameter'].max()
                min_y_value = df['Diameter'].min()
                graph = px.line(data_frame=df, x='Time', y='Diameter', width = 5000)
                graph.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=60))
                graph.add_scatter(x=df['Time'], y=df['min_value'], mode='lines', name="Min Value",
                                    line=dict(color='red', dash='dash'))
                graph.add_scatter(x=df['Time'], y=df['max_value'], mode='lines', name="Max Value",
                                    line=dict(color='red', dash='dash'))
                graph.update_layout(
                    annotations=[
                     dict(
                         x=0.01,
                         y=1,
                         showarrow = True,
                        text=f'Max value of diameter = {max_y_value}<br>Min value of diameter = {min_y_value}',
                        xref = 'paper',
                        yref = 'paper'
                        )
                    ]
                )
                html_file_name = file.replace('csv', 'html')
                graph.write_html(os.path.join(self.output_path, html_file_name))

    #Generete and save .zip file in the main path
    def create_zipfile(self):
        files_list = os.listdir(self.output_path)
        self.zip_path = 'Package.zip'

        with zipfile.ZipFile(self.zip_path, 'w') as myzip:
            for file in files_list:
                if file.endswith('html'):
                    myzip.write(os.path.join(self.output_path, file))

    #Download .zip file on the local disk
    def download_zip(self):
        files.download(self.zip_path)

new_diagram = diagram()

new_diagram.import_files_from_disc()
new_diagram.move_files_to_raw_input_data()
new_diagram.convert_txt_file_to_csv()
new_diagram.saves_csv()
new_diagram.generating_graphs()
new_diagram.create_zipfile()
new_diagram.download_zip()

Saving Nowy dokument tekstowy.txt to Nowy dokument tekstowy.txt
Wykonałem się
